In [24]:
# calclex.py

from sly import Lexer

class CalcLexer(Lexer):
    # Set of token names.   This is always required
    tokens = { NUMBER, ID, WHILE, IF, ELSE, PRINT,
               PLUS, MINUS, TIMES, DIVIDE, ASSIGN,
               EQ, LT, LE, GT, GE, NE, PUBLIC, CLASS,
               STATIC, VOID, MAIN, EXTENDS, RETURN, INT,
               BOOLEAN, NEW, TRUE, FALSE, AND}


    literals = { '(', ')', '{', '}', ';','[', ']', '!'}

    # String containing ignored characters
    ignore = ' \t'
    ignore_newline = '\n'
    ignore_comment = r'\#.*'

    # Regular expression rules for tokens
    PLUS    = r'\+'
    MINUS   = r'-'
    TIMES   = r'\*'
    DIVIDE  = r'/'
    EQ      = r'=='
    ASSIGN  = r'='
    LE      = r'<='
    LT      = r'<'
    GE      = r'>='
    GT      = r'>'
    NE      = r'!='
    AND     = r'&&'

    @_(r'\d+')
    def NUMBER(self, t):
        t.value = int(t.value)
        return t

    # Identifiers and keywords
    ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
    ID['if']    = IF
    ID['else']  = ELSE
    ID['while'] = WHILE
    ID['System.out.println'] = PRINT
    ID['class'] = CLASS
    ID['public'] = PUBLIC
    ID['static'] = STATIC
    ID['void']   = VOID
    ID['main']   = MAIN
    ID['extends']= EXTENDS 
    ID['return'] = RETURN
    ID['int']    = INT
    ID['boolean']= BOOLEAN
    ID['new']    = NEW
    ID['true']   = TRUE
    ID['false']  = FALSE


    # Line number tracking
    @_(r'\n+')
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')

    def error(self, t):
        print('Line %d: Bad character %r' % (self.lineno, t.value[0]))
        self.index += 1

if __name__ == '__main__':
    data = '''
            #Convertendo um número:
            !x = 0
            x = int(0)
        '''
    lexer = CalcLexer()
    for tok in lexer.tokenize(data):
        print(tok)

Token(type='ID', value='x', lineno=3, index=49)
Token(type='ASSIGN', value='=', lineno=3, index=51)
Token(type='BOOLEAN', value='true', lineno=3, index=53)
Token(type='ID', value='x', lineno=4, index=70)
Token(type='ASSIGN', value='=', lineno=4, index=72)
Token(type='INT', value='int', lineno=4, index=74)
Token(type='(', value='(', lineno=4, index=77)
Token(type='NUMBER', value=0, lineno=4, index=78)
Token(type=')', value=')', lineno=4, index=79)


In [2]:
!pip install sly

  Created wheel for sly: filename=sly-0.4-py3-none-any.whl size=27358 sha256=4541aff04ffe4e0a73ba9cbe008e60c7e6af632874b9f7747696df68949857b2
  Stored in directory: c:\users\san diego\appdata\local\pip\cache\wheels\4f\4a\de\a35a8fc2d21eaa16b72c1fec0abf71a10e82ba02adb1b43303
Successfully built sly


In [ ]:
from sly import Parser
from calclex import CalcLexer

class CalcParser(Parser):
    # Get the token list from the lexer (required)
    tokens = CalcLexer.tokens

    # Grammar rules and actions
    @_('expr PLUS term')
    def expr(self, p):
        return p.expr + p.term

    @_('expr MINUS term')
    def expr(self, p):
        return p.expr - p.term

    @_('term')
    def expr(self, p):
        return p.term

    @_('term TIMES factor')
    def term(self, p):
        return p.term * p.factor

    @_('term DIVIDE factor')
    def term(self, p):
        return p.term / p.factor

    @_('factor')
    def term(self, p):
        return p.factor

    @_('NUMBER')
    def factor(self, p):
        return p.NUMBER

    @_('LPAREN expr RPAREN')
    def factor(self, p):
        return p.expr

if __name__ == '__main__':
    lexer = CalcLexer()
    parser = CalcParser()

    while True:
        try:
            text = input('calc > ')
            result = parser.parse(lexer.tokenize(text))
            print(result)
        except EOFError:
            break